In [0]:
!pip install -q pydrive
!pip install -q surprise

     |████████████████████████████████| 6.5MB 2.8MB/s 


In [0]:
import pandas as pd
import joblib, surprise, random
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Mount Google Drive

In [0]:
# mount Google drive
from google.colab import drive
drive.mount('/content/drive/')

# authenticate and create the PyDrive clident
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# Load Data

In [0]:
# load reviews
file_name = 'reviews_trimmed.gz'
file_path = F'/content/drive/My Drive/Insight/Colab_Data/{file_name}'
reviews_df = joblib.load(file_path)
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9321847 entries, 0 to 99999
Data columns (total 7 columns):
user_id       int64
book_id       int64
rating        int64
read_at       object
started_at    object
Chunk         object
book_id2      int64
dtypes: int64(4), object(3)
memory usage: 569.0+ MB


# Generate and Save Datasets

## Parameters

In [0]:
# define rating scale
reader = surprise.Reader(rating_scale=(1, 5))

# column names to use in building the collaborative filtering models
col_names = ['book_id', 'user_id', 'rating']

# construct list from dataframe
reviews_list = reviews_df[col_names].values.tolist()

# shuffle dataframe values
random.shuffle(reviews_list) # shuffles in-place

## Generate Training Sets of Different Sizes

In [0]:
for num_reviews in [1E4, 5E4, 1E5, 5E5, 1E6, 5E6]:
  print('Number of reviews in dataset =', num_reviews)

  # only use the number of reviews specified
  reviews_set = pd.DataFrame(reviews_list[:int(num_reviews)], columns=col_names)

  # convert to dataset
  reviews = surprise.Dataset.load_from_df(reviews_set, reader)

  # upload training set to Google drive
  print('Uploading training data...')
  train_name = 'trainingReviews_' + str(int(num_reviews)) + '.joblib'
  train_path = F'/content/drive/My Drive/Insight/Colab_Data/{train_name}'
  joblib.dump(reviews, train_path)

  print('...done!\n')

Number of reviews in dataset = 10000.0
Uploading training data...
...done!

Number of reviews in dataset = 50000.0
Uploading training data...
...done!

Number of reviews in dataset = 100000.0
Uploading training data...
...done!

Number of reviews in dataset = 500000.0
Uploading training data...
...done!

Number of reviews in dataset = 1000000.0
Uploading training data...
...done!

Number of reviews in dataset = 5000000.0
Uploading training data...
...done!

